In [23]:
import requests
import json
import configparser
# from pprint import pprint

ImportError: No module named 'pyspark'

In [2]:
config = configparser.ConfigParser()
config.read('../../../gnip_auth.ini')
api_user = config['GNIP_API']['username']
api_pass = config['GNIP_API']['password']

**Using GNIP API search filters**

In [3]:
class GnipData():
    
    def __init__(maxResults, geolocation, fromDate, toDate):
        self.maxResults = maxResults
        self.geolocation = geolocation
        self.fromDate = fromDate
        self.toDate = toDate
        self._cred = (api_user, api_pass)
        self.url = "https://gnip-api.twitter.com/search/" \
        "fullarchive/accounts/greg-students/prod.json"
    
    
    def fetchTweets(queries):
        params = {'fromDate':'201401010000',
                  'toDate': '201703200000',
                  'maxResults':self.maxResults}
        
        for query in queries:
            params['query'] = query
            
            response = requests.get(self.url,
                                    params=params,
                                    auth=self._cred)

            queueKafka(response.json()['results'])
            
            while ('next' in response.json().keys()):
                params['next'] = response.json()['next']
                
                response = requests.get(url,
                                        params=params,
                                        auth=self._cred)

                queueKafka(response.json()['results'])

    
    def queueKafka(json_data):
        # queue json data in kafka
        pass


In [4]:
class SparkProcessing():
    
    def __init__(snopes_urls):
        self.urls = snopes_urls
    
    
    def runSpark():
        getQueries(self.urls)
    
    
    def getQueries(urls):
        # spark processing on urls to get keywords
        pass

In [5]:
# process_urls = SparkProcessing(snopes_urls)
# queries = process_urls.runSpark()

# search = GnipData(100, 'Boulder', blah, blah)
# search.fetchTweets(queries)

In [6]:
def fetchTweets(query, maxResults=500):
    # search API url
    url = "https://gnip-api.twitter.com/search/" \
    "fullarchive/accounts/greg-students/prod.json"
    
    # storing auth credentials in a hidden variable
    _cred=(api_user, api_pass)
    
    # setting parameters to append to the url
    params = {'query':query,
              'fromDate':'201401010000',
              'toDate': '201703200000',
              'maxResults': 500}
    
    # making a GET request to the API
    response = requests.get(url,
                            params=params,
                            auth=_cred)
    
    # storing the number of tweets returned
    reslen = len(response.json()['results'])
    
    # checking if more tweets are available
    while ('next' in response.json().keys()):

        yield reslen
        
        params['next'] = response.json()['next']

        response = requests.get(url,
                                params=params,
                                auth=_cred)

        reslen += len(response.json()['results'])

In [7]:
# using search API word locality filter
for num_tweets in fetchTweets("obama thank you"):
    print(num_tweets)


500
1000
1500
2000
2500
3000
3500
4000


KeyboardInterrupt: 

In [25]:
import requests
import json
import configparser
from pykafka import KafkaClient

##parameters for the search
url ="https://gnip-api.twitter.com/search/fullarchive/accounts/greg-students/prod.json"
parameters = { 'query':'gnip', 'maxResults':'10',
'fromDate':'201701010000', 'toDate':'201701050000'}

_cred=(api_user, api_pass)

##reading credentails file
config = configparser.ConfigParser()
config.read('credentials.ini')

##connecting to the kafka client
client = KafkaClient("localhost:9092")

##selcting the topic
topic = client.topics[b'test']

response = requests.get(url,
                        auth=_cred,
                        params=parameters)

res = response.json()['results']
consumer = topic.get_simple_consumer()
##writing into topic
with topic.get_producer() as producer:
    producer.produce(bytes(json.dumps(res).encode("utf-8")))
    msg = consumer.consume()
    print(msg.value)

b'[{"truncated": false, "place": null, "retweet_count": 0, "favorite_count": 7, "quote_count": 0, "in_reply_to_status_id_str": null, "contributors": null, "source": "<a href=\\"http://twitter.com\\" rel=\\"nofollow\\">Twitter Web Client</a>", "favorited": false, "in_reply_to_user_id_str": "2249227634", "retweeted": false, "filter_level": "low", "user": {"profile_background_image_url_https": "https://abs.twimg.com/images/themes/theme1/bg.png", "profile_link_color": "1DA1F2", "default_profile_image": false, "profile_text_color": "333333", "profile_banner_url": "https://pbs.twimg.com/profile_banners/1425058188/1473754667", "profile_use_background_image": true, "profile_image_url": "http://pbs.twimg.com/profile_images/778273590684844032/lOZQ8YbG_normal.jpg", "profile_background_color": "C0DEED", "profile_background_tile": false, "verified": false, "time_zone": "Pacific Time (US & Canada)", "derived": {"locations": [{"full_name": "Leeds, England, United Kingdom", "geo": {"type": "point", "c